In [1]:
import numpy as np
from collections import defaultdict
import nltk 
import pandas as pd
import os 
import json
import io
import gzip

The code shown below is from 

In [14]:
# read in a gz file
def readGz(fname):
    gz = gzip.open(fname, 'rb')
    f = io.BufferedReader(gz)
    data = []
    rows = 0

    for l in f.readlines():
        rows = rows + 1
        data.append(eval(l))
        if rows == 10000:
            break;
        
    gz.close()
    return data
#     for l in gzip.open(f):
#         yield eval(l)

In [15]:
%%time
#parsing in the data, takes time
places = readGz("places.clean.json.gz")
users = readGz("users.clean.json.gz")
reviews = readGz("reviews.clean.json.gz")

Wall time: 3min 22s


In [16]:
places = pd.DataFrame(places)
users = pd.DataFrame(users)
reviews = pd.DataFrame(reviews)

In [28]:
places.head()

,address,closed,gPlusPlaceId,gps,hours,name,phone,price
0,"[2615 Angler Ave, Hemet, CA 92545]",False,104699454385822125632,"[33.703804, -117.003209]","[[Monday, [['6:30 am--4:15 pm']]], [Tuesday, [...",Diamond Valley Lake Marina,(951) 926-7201,None
1,"[Parole, Annapolis, MD]",False,103054478949000078829,"[38.979759, -76.547538]",None,Blue Ribbon Cleaners,(410) 266-6123,None
2,"[ул. Тутаева, 1, Nazran, Ingushetia, Russia, 3...",False,109810290098030327104,"[43.22776, 44.762726]","[[Monday, [['9:30 am--9:00 pm']]], [Tuesday, [...",Portofino,8 (963) 173-38-38,None
3,"[5322 W 26th St, Sioux Falls, SD 57106]",False,100327153115986850675,"[43.529494, -96.792244]","[[Monday, [['11:00 am--10:00 pm']]], [Tuesday,...",T C's Referee Sports Bar,(605) 361-2208,$$
4,"[Jl. Angkatan 45, Kompleks Palembang Square, S...",False,103368487323937936043,"[-2.976256, 104.742662]","[[Monday, [['8:00 am--9:00 pm']]], [Tuesday, [...",Carrefour - Palembang Square,(0711) 359918,None


In [27]:
reviews.head()

,categories,gPlusPlaceId,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime
0,[Giải Trí - Café],108103314380004200232,100000010817154263736,3.0,Chất lượng tạm ổn,"Jul 1, 2013",an lam,1.372687e+09
1,[Turkish Cuisine],102194128241608748649,100000013500285534661,5.0,Wc si temiz duzenli..,"Jul 21, 2012",HALİL TURGUT,1.342871e+09
2,"[Fishing, Pond Fish Supplier, Seafood Market]",101409858828175402384,100000021336848867366,5.0,何回も私は予定に休みがセルバに行ったので覚えて見て、分かります❗,"Jan 25, 2014",森田さとこ,1.390654e+09
3,[Museum],101477177500158511502,100000021336848867366,5.0,今度は予定に休みが登米市に行きたい❗☀😅🌌 楽しいに日帰りに登米の見学の観光(*^)(*^-...,"Jan 8, 2014",森田さとこ,1.389188e+09
4,[Police],106994170641063333085,100000021336848867366,4.0,気仙沼警察署に移転中に絆 👮🐎☺🙋🚓頑張ろう❗,"Jan 23, 2014",森田さとこ,1.390486e+09


In [17]:
users.head()

,currentPlace,education,gPlusUserId,jobs,previousPlaces,userName
0,"[Thành phố Hồ Chí Minh, Việt Nam, [[], 1082309...","[[[], [], [], [], [], 6], [[Đại học Kiến trúc ...",100000010817154263736,"[[Tổng công ty IDICO, Chuyên viên Kỹ thuật XD,...","[[tp. Nam Định, Nam Định, Việt Nam, [[], 20420...",an lam
1,"[Adana, [[], 370000000, 353213330, 1]]","[[[], [], [], [], [], 6], [[Anadolu University...",100000013500285534661,"[[Mustafa Nevzat İlaç San. A.Ş., Rebrezant, []...",None,HALİL TURGUT
2,None,"[[[], [], [], [], [], 6], []]",100000021336848867366,None,None,森田さとこ
3,None,"[[[], [], [], [], [], 6], []]",100000030557048145331,None,None,amey kore
4,None,"[[[], [], [], [], [], 6], []]",100000032416892623125,None,None,william spindler


In [24]:
reviews.merge(users, how = 'outer') #, #= "gPlusUserId")

,categories,gPlusPlaceId,gPlusUserId,rating,reviewText,reviewTime,reviewerName,unixReviewTime,currentPlace,education,jobs,previousPlaces,userName
0,[Giải Trí - Café],108103314380004200232,100000010817154263736,3.0,Chất lượng tạm ổn,"Jul 1, 2013",an lam,1.372687e+09,"[Thành phố Hồ Chí Minh, Việt Nam, [[], 1082309...","[[[], [], [], [], [], 6], [[Đại học Kiến trúc ...","[[Tổng công ty IDICO, Chuyên viên Kỹ thuật XD,...","[[tp. Nam Định, Nam Định, Việt Nam, [[], 20420...",an lam
1,[Turkish Cuisine],102194128241608748649,100000013500285534661,5.0,Wc si temiz duzenli..,"Jul 21, 2012",HALİL TURGUT,1.342871e+09,"[Adana, [[], 370000000, 353213330, 1]]","[[[], [], [], [], [], 6], [[Anadolu University...","[[Mustafa Nevzat İlaç San. A.Ş., Rebrezant, []...",None,HALİL TURGUT
2,"[Fishing, Pond Fish Supplier, Seafood Market]",101409858828175402384,100000021336848867366,5.0,何回も私は予定に休みがセルバに行ったので覚えて見て、分かります❗,"Jan 25, 2014",森田さとこ,1.390654e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
3,[Museum],101477177500158511502,100000021336848867366,5.0,今度は予定に休みが登米市に行きたい❗☀😅🌌 楽しいに日帰りに登米の見学の観光(*^)(*^-...,"Jan 8, 2014",森田さとこ,1.389188e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
4,[Police],106994170641063333085,100000021336848867366,4.0,気仙沼警察署に移転中に絆 👮🐎☺🙋🚓頑張ろう❗,"Jan 23, 2014",森田さとこ,1.390486e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
5,[Theme Park],107443380765688783479,100000021336848867366,5.0,久しぶりに仙台市八木山ベニーランドに乗りたい❗,"Jan 19, 2014",森田さとこ,1.390193e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
6,[Shopping Mall],107998775704210253660,100000021336848867366,5.0,私は今度は休みがザ・モール★の映画に行って来ます。,"Jan 27, 2014",森田さとこ,1.390826e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
7,[Art Museum],111367239248458923884,100000021336848867366,4.0,イベントの光のページェント が素晴らしい❗🔍 私はとても仮面ライダーが素晴らしいが良いがです🎵ネ。,"Jan 22, 2014",森田さとこ,1.390446e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
8,[Science Museum],111951491217608644315,100000021336848867366,4.0,ヒマ👽休みが仙台市科学館 に行って見ます❗,"Jan 22, 2014",森田さとこ,1.390388e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
9,[Grocery Store],113899620351618240699,100000021336848867366,5.0,久しぶりに青葉城に行きたい❗,"Feb 13, 2014",森田さとこ,1.392295e+09,None,"[[[], [], [], [], [], 6], []]",None,None,森田さとこ
